# Level 3: Advanced Agent Capabilities with Prompt Chaining and ReAct Agent

Building on the simple agent introduced in [Level 2](Level2_simple_agent_with_websearch.ipynb), this tutorial continues the agent-focused section of our series by introducing techniques that make the agent smarter and more autonomous: **Prompt Chaining** and the **ReAct (Reasoning + Acting) framework**. These approaches allow the agent to complete multi-step tasks, dynamically choose tools, and adjust its behavior based on context.

- **Prompt Chaining** connects multiple prompts into a coherent sequence, allowing the agent to maintain context and perform multi-step reasoning across tool invocations. 
- **ReAct Agent** combines reasoning and acting steps in a loop, enabling the agent to make decisions, use tools dynamically, and adapt based on intermediate results. 

## Overview

In this notebook, you'll explore three agent configurations:
1. **Simple Agent (Baseline)** – Uses a single web search tool.
2. **Prompt Chaining** – Performs structured, multi-step reasoning by chaining prompts and responses.
3. **ReAct Agent** – Dynamically plans and executes actions using a loop of reasoning and tool use.


## Prerequisites

Before starting this notebook, ensure that you have:
- Followed the instructions in the [Setup Guide](./Level0_getting_started_with_Llama_Stack.ipynb) notebook. 
- A Tavily API key. It is critical for this notebook to run correctly. You can register for one at [https://tavily.com/](https://tavily.com/).

## 1. Setting Up this Notebook
We will start with a few imports needed for this demo only.

In [1]:
!pip install geocoder 


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade "pydantic>=2.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.22
    Uninstalling pydantic-1.10.22:
      Successfully uninstalled pydantic-1.10.22
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codeflare-sdk 0.29.0 requires pydantic<2, but you have pydantic 2.11.7 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('..') 
from src.client_tools import get_location

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](./Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [4]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
# model_id = "deepseek"
model_id = "granite"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: granite
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 2048}
	stream: False


## 2. Simple Agent (Baseline)
Same agent setup as [Level 2 notebook](Level2_simple_agent_with_websearch.ipynb). 

In [5]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful websearch assistant. When you are asked to search the latest you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://llamastack:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/1af836a3-49b3-4545-8af5-350c77dc3e6a/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/1af836a3-49b3-4545-8af5-350c77dc3e6a/session/e7392039-1c27-491e-8395-4c7c3e42bbdc/turn "HTTP/1.1 200 OK"



Processing user query: Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': "current weather risks in user's location"}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': "current weather risks in user's location",
│   'top_k': [
│   │   {
│   │   │   'title': "Weather in user's location",
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Zebedielas Location', 'region': 'Limpopo', 'country': 'South Africa', 'lat': -24.4167, 'lon': 29.3833, 'tz_id': 'Africa/Johannesburg', 'localtime_epoch': 1756084799, 'localtime': '2025-08-25 03:19'}, 'current': {'last_updated_epoch': 1756084500, 'last_updated': '2025-08-25 03:15', 'temp_c': 18.1, 'temp_f': 64.5, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 43, 'wind_dir': 'NE', 'pressure_mb': 1017.0, 'pressure_in': 30.04, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 37, 'cloud': 0, 'feelslike_c': 18.1, 'feelslike_f': 64.5, 'windchill_c': 18.1, 'windchill_f': 64.5, 'heatindex_c': 18.1, 'heatindex_f': 64.5, 'dewpoint_c': 3.1, 'dewpoint_f': 37.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 9.4, 'gust_kph': 15.1}}",
│   │   │   'score': 0.9904004,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://www.noaa.gov/weather-prediction-center',
│   │   │   'title': 'Weather Prediction Center (WPC) Home Page - NOAA',
│   │   │   'content': "Check out the HeatRisk page to understand potential heat impacts. headline2. Explore WPC's experimental Probabilistic Precipitation Portal. headline3",
│   │   │   'score': 0.98567,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://digital.weather.gov/',
│   │   │   'title': 'National Weather Service - Graphical Forecast',
│   │   │   'content': 'Missing: risks 25/08/2025',
│   │   │   'score': 0.98359,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://www.weather.gov/media/directives/010_pdfs/pd01004001curr.pdf',
│   │   │   'title': '[PDF] national weather service instruction 10-401',
│   │   │   'content': 'A. Page 7. 7. NWSI 10-401 AUGUST 25, 2025. WFO may also highlight particularly dangerous RFW conditions for the public, if this message is closely coordinated',
│   │   │   'score': 0.97693,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://www.wpc.ncep.noaa.gov/index.shtml',
│   │   │   'title': 'Weather Prediction Center (WPC) Home Page',
│   │   │   'content': 'Short Range Forecast Discussion NWS Weather Prediction Center College Park MD 400 PM EDT Sat Aug 23 2025. Valid 00Z Sun Aug 24 2025 - 00Z Tue Aug 26 2025',
│   │   │   'score': 0.97356,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the latest weather information, there are no immediate weather-related risks in your area that could disrupt network connectivity or system availability. The current weather in your location is clear with a temperature of 64.5°F (18.1°C). There are no significant wind speeds, humidity is low at 37%, and there's no precipitation expected. 

However, for the most accurate and up-to-date information, I would recommend checking the National Weather Service's website or app. They provide real-time updates and forecasts that can help you stay informed about any potential weather changes.

========== Query processing completed ========== 



### Output Analysis

In this example, since the agent is unaware of the users location, it hallucinates one and generates an incorrect search query. This misidentification leads to inaccurate information about potential weather-related risks.

This is where Prompt Chaining comes in. Prompt chaining allows the agent to:
1. Maintain context across multiple queries
2. Chain multiple tools together
3. Use previous interactions to inform current decisions

Let’s see how prompt chaining can improve the accuracy of the response.

## 3. Prompt chaining with websearch tool and client tool

In this section, we demonstrate a more sophisticated use case that combines the use of two tools: location detection and web search.

1. **Automatic Location Detection**: Use the `get_location` client tool to automatically determine the user's current location.
2. **Contextual Search**: Leverage the detected location to formulate the correct websearch query.

For example, when a user asks "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:
- First detect the user's current location using `get_location`.
- Then use that location to search for nearby weather-related risks.
- Finally, present a comprehensive response.

This demonstrates how the builtin websearch tool and custom client tools can work together to provide intelligent, context-aware responses without requiring explicit location input from the user.

In [6]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When you are asked to search the latest news, you MUST use the websearch tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Where am I?",
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("prompt-chaining-session")  # for prompt chaining, queries must share the same session_id.
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://llamastack:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/c4a3b3df-d11c-4c0f-bb67-b7d537d47389/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/c4a3b3df-d11c-4c0f-bb67-b7d537d47389/session/322df2d5-b185-4d2d-a67b-9d47e6910efe/turn "HTTP/1.1 200 OK"



Processing user query: Where am I?


INFO:geocoder.base:Requested http://ipinfo.io/json
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/c4a3b3df-d11c-4c0f-bb67-b7d537d47389/session/322df2d5-b185-4d2d-a67b-9d47e6910efe/turn/9be0d025-527e-418e-b8bc-e3d641a907e9/resume "HTTP/1.1 200 OK"



---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Frankfurt am Main, Hesse, DE'

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/c4a3b3df-d11c-4c0f-bb67-b7d537d47389/session/322df2d5-b185-4d2d-a67b-9d47e6910efe/turn "HTTP/1.1 200 OK"



---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
You are currently in Frankfurt am Main, Hesse, Germany.

========== Query processing completed ========== 


Processing user query: Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'Frankfurt am Main immediate weather risks'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'Frankfurt am Main immediate weather risks',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Frankfurt am Main, Germany',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Frankfurt Am Main', 'region': 'Hessen', 'country': 'Germany', 'lat': 50.1167, 'lon': 8.6833, 'tz_id': 'Europe/Berlin', 'localtime_epoch': 1756084817, 'localtime': '2025-08-25 03:20'}, 'current': {'last_updated_epoch': 1756084500, 'last_updated': '2025-08-25 03:15', 'temp_c': 10.0, 'temp_f': 50.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 27, 'wind_dir': 'NNE', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 76, 'cloud': 0, 'feelslike_c': 10.1, 'feelslike_f': 50.2, 'windchill_c': 11.9, 'windchill_f': 53.3, 'heatindex_c': 11.6, 'heatindex_f': 52.8, 'dewpoint_c': 3.8, 'dewpoint_f': 38.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 4.9, 'gust_kph': 7.8}}",
│   │   │   'score': 0.7645227,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://www.weathertab.com/en/d/e/08/federal-republic-of-germany/60596-frankfurt-am-main/',
│   │   │   'title': 'August 2025 Daily Weather Forecast for 60596 (Frankfurt am Main)',
│   │   │   'content': '60596 (Frankfurt am Main), Daily Weather Forecast for August 2025 ; 8. Fri. 60%. 70 to 80 °F 51 to 61 °F 18 to 28 °C 8 to 18 °C · Sunrise 6:04AM. Sunset 8:56PM.',
│   │   │   'score': 0.7500592,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://www.weather2travel.com/germany/frankfurt/august/',
│   │   │   'title': 'Frankfurt weather in August 2025 | Germany: How hot?',
│   │   │   'content': "# Frankfurt weather in August 2025 Expect  **daytime maximum temperatures of 23°C** in **Frankfurt, Germany** in **August** based on long-term weather averages. The August weather guide for Germany (Frankfurt) shows long term weather averages processed from data supplied by **CRU (University of East Anglia)**, the **Met Office** & the **Netherlands Meteorological Institute**. There are normally **7 hours** of bright sunshine each day in **Frankfurt** in **August** - that's **46% of daylight hours**. There are usually **14 days** with some rain in **Frankfurt** in **August** and the average monthly rainfall is **68mm**. ## Frankfurt August sunrise & sunset times Browse the sunrise and sunset times for Frankfurt in August 2025. * Summer holidays * Holiday offers * Airport parking deals More holidays",
│   │   │   'score': 0.6633424,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://world-weather.info/forecast/germany/frankfurt_am_main/august-2025/',
│   │   │   'title': 'Weather in Frankfurt am Main in August 2025 (Hesse)',
│   │   │   'content': "Frankfurt am Main Weather Forecast for August 2025 is based on long term prognosis and previous years' statistical data.",
│   │   │   'score': 0.61196244,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://en.climate-data.org/europe/germany/hesse/frankfurt-am-main-447/t/august-8/',
│   │   │   'title': 'Weather Frankfurt am Main in August 2025 - Climate-Data.org',
│   │   │   'content': 'Weather Frankfurt am Main in August 2025: Temperature & Climate *   Overview: Frankfurt am Main Weather and Climate in August The average warmest day in Frankfurt am Main during August is 02.08, with a high of 26.5°C | 79.8°F. #### What day in August has the highest temperature in Frankfurt am Main? In August, the warmest day in Frankfurt am Main is 02.08, with a temperature of 26.5°C | 79.8°F. #### During August, which 10-day stretch has the highest temperatures in Frankfurt am Main? #### During August, which 10-day stretch has the lowest temperatures in Frankfurt am Main? #### What day in August has the highest humidity in Frankfurt am Main? #### W


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
According to the latest weather forecasts, there are no immediate weather-related risks in Frankfurt am Main that could disrupt network connectivity or system availability. The weather is currently clear with a temperature of 10.0°C (50.0°F). There are no significant weather warnings or alerts for the area.

========== Query processing completed ========== 



### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

Here is a walkthrough of how the ReAct agent will tackle this same "weather near me" problem:

When asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:

1. **Reason** that it needs to get location information first.
2. **Act** by calling the `get_location` client tool.
3. **Observe** the location result.
4. **Reason** that it now needs to search for weather in that location.
5. **Act** by calling the `websearch` tool with observed location.
6. **Observe** and processes the search results into a final answer. 

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [7]:
agent = ReActAgent(
            client=client,
            model=model_id,
            tools=[get_location, "builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )
user_prompts = [
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

INFO:httpx:HTTP Request: GET http://llamastack:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://llamastack:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/3598fb2a-bc17-4b0e-9023-e7e1cc337494/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/3598fb2a-bc17-4b0e-9023-e7e1cc337494/session/3579ef8f-dc2c-426e-9105-74b049b77e04/turn "HTTP/1.1 200 OK"



Processing user query: Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?


INFO:geocoder.base:Requested http://ipinfo.io/json
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/3598fb2a-bc17-4b0e-9023-e7e1cc337494/session/3579ef8f-dc2c-426e-9105-74b049b77e04/turn/71c140b0-61d0-44cb-a3a2-d87ad01a043a/resume "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/tool-runtime/invoke "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/3598fb2a-bc17-4b0e-9023-e7e1cc337494/session/3579ef8f-dc2c-426e-9105-74b049b77e04/turn/71c140b0-61d0-44cb-a3a2-d87ad01a043a/resume "HTTP/1.1 200 OK"



---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
    "thought": "I need to get the user's current location and then check for any weather-related risks in that area. I will use the `get_location` tool to find the user's location and then the `web_search` tool to check for any weather-related risks.",
    "action": {
        "tool_name": "get_location",
        "tool_params": [{"name": "query", "value": "user's current location"}]
    },
    "answer": null
}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Frankfurt am Main, Hesse, DE'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Now that I have the user's location, I will use the `web_search` tool to check for any weather-related risks in Frankfurt am Main, Hesse, DE.",
    "action": {
        "tool_name": "web_search",
        "tool_params": [{"name": "query", "value": "weather-related risks in Frankfurt am Main, Hesse, DE"}]
    },
    "answer": null
}


---------- 📍 Step 4: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks in Frankfurt am Main, Hesse, DE',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Frankfurt am Main, Hesse, DE',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Frankfurt Am Main', 'region': 'Hessen', 'country': 'Germany', 'lat': 50.1167, 'lon': 8.6833, 'tz_id': 'Europe/Berlin', 'localtime_epoch': 1756084814, 'localtime': '2025-08-25 03:20'}, 'current': {'last_updated_epoch': 1756084500, 'last_updated': '2025-08-25 03:15', 'temp_c': 10.0, 'temp_f': 50.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 27, 'wind_dir': 'NNE', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 71, 'cloud': 0, 'feelslike_c': 10.1, 'feelslike_f': 50.2, 'windchill_c': 11.9, 'windchill_f': 53.3, 'heatindex_c': 11.6, 'heatindex_f': 52.8, 'dewpoint_c': 3.8, 'dewpoint_f': 38.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 4.9, 'gust_kph': 7.8}}",
│   │   │   'score': 0.7418937,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://world-weather.info/forecast/germany/frankfurt_am_main/august-2025/',
│   │   │   'title': 'Weather in Frankfurt am Main in August 2025 (Hesse)',
│   │   │   'content': 'Missing: related risks DE',
│   │   │   'score': 0.70944256,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://www.weather2travel.com/germany/frankfurt/august/',
│   │   │   'title': 'Frankfurt weather in August 2025 | Germany: How hot?',
│   │   │   'content': "# Frankfurt weather in August 2025 Expect  **daytime maximum temperatures of 23°C** in **Frankfurt, Germany** in **August** based on long-term weather averages. The August weather guide for Germany (Frankfurt) shows long term weather averages processed from data supplied by **CRU (University of East Anglia)**, the **Met Office** & the **Netherlands Meteorological Institute**. There are normally **7 hours** of bright sunshine each day in **Frankfurt** in **August** - that's **46% of daylight hours**. There are usually **14 days** with some rain in **Frankfurt** in **August** and the average monthly rainfall is **68mm**. ## Frankfurt August sunrise & sunset times Browse the sunrise and sunset times for Frankfurt in August 2025. * Summer holidays * Holiday offers * Airport parking deals More holidays",
│   │   │   'score': 0.539833,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://www.timeanddate.com/weather/@3220968/ext',
│   │   │   'title': 'Frankfurt am Main, Hesse, Germany 14 day weather forecast',
│   │   │   'content': '2 Week Extended Forecast in Frankfurt am Main, Hesse, Germany. Increasing cloudiness. Mostly cloudy. Isolated tstorms late.',
│   │   │   'score': 0.5201313,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'url': 'https://weatherspark.com/h/y/61302/2025/Historical-Weather-during-2025-in-Frankfurt-am-Main-Hesse-Germany',
│   │   │   'title': '2025 Weather History in Frankfurt am Main Hesse, Germany',
│   │   │   'content': 'Frankfurt am Main Temperature History 2025 Link  Download  Compare  Averages  History:202520242023202220212020201920182017    The daily range of reported temperatures (gray bars) and 24-hour highs (red ticks) and lows (blue ticks), placed over the daily average high (faint red line) and low (faint blue line) temperature, with 25th to 75th and 10th to 90th percentile bands. Hourly Temperature in 2025 in Frankfurt am Main Link  Download  Compare  Averages  History:202520242023202220212020201920182017    *frigid*   *15°F* *freezing*   *32°F* *very cold*   *45°F* *cold*   *55°F* *cool*   *65°F* *comfortable*   *75°F* *warm*   *85°F* *hot*   *95°F* *sweltering*  The hourly reported temperature, color coded into bands. Hours of Daylight and Twilight in 2025 in Frankfurt am Main Link  Download  Compar


---------- 📍 Step 5: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Based on the search results, there are no immediate weather-related risks in Frankfurt am Main, Hesse, DE that could disrupt network connectivity or system availability. The weather is mostly cloudy with isolated thunderstorms late.",
    "action": null,
    "answer": "There are no immediate weather-related risks in Frankfurt am Main, Hesse, DE that could disrupt network connectivity or system availability. The weather is mostly cloudy with isolated thunderstorms late."
}

========== Query processing completed ========== 



## Key Takeaways
- This notebook demonstrated how to build more capable agents using Prompt Chaining and the ReAct framework.
- It showed how agents can maintain context across multiple steps and perform structured, multi-step reasoning.
- It highlights how ReAct enables dynamic tool selection and adaptive decision-making based on intermediate results.
- These techniques enhance agent autonomy and make them more suitable for complex operational tasks.

For further extensions, continue exploring in the next notebook: [RAG Agents](Level4_RAG_agent.ipynb).

#### Any Feedback?

If you have any feedback on this or any other notebook in this demo series we'd love to hear it! Please go to https://www.feedback.redhat.com/jfe/form/SV_8pQsoy0U9Ccqsvk and help us improve our demos. 